In [ ]:
import datetime
import tkinter as tk
from tkinter import messagebox
from abc import ABC, abstractmethod

# UTILITÁRIOS

def mensagem(msg):
    messagebox.showinfo("Mensagem", msg)

# MODELOS

class Historico:
    def __init__(self):
        self.transacoes = []

    def adicionar_transacao(self, tipo, valor):
        self.transacoes.append({
            "tipo": tipo,
            "valor": valor,
            "data": datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        })

    def mostrar(self):
        if not self.transacoes:
            return "Nenhuma transação realizada."
        return "\n".join([f"{t['data']} - {t['tipo']}: R$ {t['valor']:.2f}" for t in self.transacoes])

class Conta:
    def __init__(self, agencia, numero, cliente):
        self.agencia = agencia
        self.numero = numero
        self.cliente = cliente
        self.saldo = 0.0
        self.historico = Historico()

    def depositar(self, valor):
        if valor > 0:
            self.saldo += valor
            self.historico.adicionar_transacao("Depósito", valor)
            mensagem(f"Depósito de R$ {valor:.2f} realizado com sucesso.")
            return True
        mensagem("Valor inválido para depósito.")
        return False

    def sacar(self, valor):
        if 0 < valor <= self.saldo:
            self.saldo -= valor
            self.historico.adicionar_transacao("Saque", valor)
            mensagem(f"Saque de R$ {valor:.2f} realizado com sucesso.")
            return True
        mensagem("Saque não autorizado. Verifique saldo e limite.")
        return False

    def extrato(self):
        extrato = self.historico.mostrar()
        extrato += f"\nSaldo atual: R$ {self.saldo:.2f}\n"
        return extrato

class ContaCorrente(Conta):
    def __init__(self, agencia, numero, cliente, limite_saque=500, limite_saques_dia=3):
        super().__init__(agencia, numero, cliente)
        self.limite_saque = limite_saque
        self.limite_saques_dia = limite_saques_dia
        self.saques_realizados = 0
        self.data_ultimo_saque = None

    def sacar(self, valor):
        hoje = datetime.date.today()
        if self.data_ultimo_saque != hoje:
            self.saques_realizados = 0
            self.data_ultimo_saque = hoje

        if self.saques_realizados >= self.limite_saques_dia:
            mensagem("Limite diário de saques atingido.")
            return False

        if valor > self.limite_saque:
            mensagem(f"Valor excede o limite de saque: R$ {self.limite_saque:.2f}.")
            return False

        if super().sacar(valor):
            self.saques_realizados += 1
            return True
        return False

class Cliente:
    def __init__(self, nome, cpf, nascimento, endereco):
        self.nome = nome
        self.cpf = cpf
        self.nascimento = nascimento
        self.endereco = endereco
        self.contas = []

    def adicionar_conta(self, conta):
        self.contas.append(conta)

    def recuperar_conta(self):
        return self.contas[0] if self.contas else None

class Banco:
    def __init__(self):
        self.clientes = []
        self.contas = []
        self.numero_conta = 1

    def buscar_cliente(self, cpf):
        return next((c for c in self.clientes if c.cpf == cpf), None)

    def criar_cliente(self, nome, cpf, nascimento, endereco):
        if self.buscar_cliente(cpf):
            mensagem("CPF já cadastrado.")
            return
        cliente = Cliente(nome, cpf, nascimento, endereco)
        self.clientes.append(cliente)
        mensagem("Cliente cadastrado com sucesso.")

    def criar_conta_corrente(self, cpf):
        cliente = self.buscar_cliente(cpf)
        if not cliente:
            mensagem("Cliente não encontrado.")
            return
        conta = ContaCorrente("0001", str(self.numero_conta), cliente)
        cliente.adicionar_conta(conta)
        self.contas.append(conta)
        self.numero_conta += 1
        mensagem("Conta criada com sucesso.")

    def depositar(self, cpf, valor):
        cliente = self.buscar_cliente(cpf)
        if not cliente:
            mensagem("Cliente não encontrado.")
            return
        conta = cliente.recuperar_conta()
        if not conta:
            mensagem("Conta não encontrada.")
            return
        conta.depositar(valor)

    def sacar(self, cpf, valor):
        cliente = self.buscar_cliente(cpf)
        if not cliente:
            mensagem("Cliente não encontrado.")
            return
        conta = cliente.recuperar_conta()
        if not conta:
            mensagem("Conta não encontrada.")
            return
        conta.sacar(valor)

    def ver_extrato(self, cpf):
        cliente = self.buscar_cliente(cpf)
        if not cliente:
            mensagem("Cliente não encontrado.")
            return
        conta = cliente.recuperar_conta()
        if not conta:
            mensagem("Conta não encontrada.")
            return
        messagebox.showinfo("Extrato", conta.extrato())

# INTERFACE TKINTER

banco = Banco()

root = tk.Tk()
root.title("Sistema Bancário Lopes")
root.geometry("400x400")

# Entradas
entry_nome = tk.Entry(root)
entry_cpf = tk.Entry(root)
entry_nasc = tk.Entry(root)
entry_end = tk.Entry(root)
entry_valor = tk.Entry(root)

entry_nome.insert(0, "Nome")
entry_cpf.insert(0, "CPF")
entry_nasc.insert(0, "Nascimento")
entry_end.insert(0, "Endereço")
entry_valor.insert(0, "Valor")

entry_nome.pack()
entry_cpf.pack()
entry_nasc.pack()
entry_end.pack()
entry_valor.pack()

btn_criar_cliente = tk.Button(root, text="Criar Cliente", command=lambda: banco.criar_cliente(entry_nome.get(), entry_cpf.get(), entry_nasc.get(), entry_end.get()))
btn_criar_conta = tk.Button(root, text="Criar Conta", command=lambda: banco.criar_conta_corrente(entry_cpf.get()))
btn_depositar = tk.Button(root, text="Depositar", command=lambda: banco.depositar(entry_cpf.get(), float(entry_valor.get())))
btn_sacar = tk.Button(root, text="Sacar", command=lambda: banco.sacar(entry_cpf.get(), float(entry_valor.get())))
btn_extrato = tk.Button(root, text="Ver Extrato", command=lambda: banco.ver_extrato(entry_cpf.get()))

btn_criar_cliente.pack(pady=2)
btn_criar_conta.pack(pady=2)
btn_depositar.pack(pady=2)
btn_sacar.pack(pady=2)
btn_extrato.pack(pady=2)

root.mainloop()
